In [ ]:
import os
import cv2 #OpenCV 
import numpy as np
from keras.preprocessing.image import array_to_img
from tkinter import *
from collections import Counter
from keras.models import load_model

#load the trained network
model = load_model('face_recognition_model.h5')

face_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_frontalface_default.xml')


#main directory with data - containing subfolders whose names means the person's name
data_dir = 'data'
persons = os.listdir(data_dir) #get all subdirectories - one folder means one person

#create list of numbers from 0 to label each person
labels = []
for i in range(0, len(persons)):
    labels.append(i)

    
#create a person-label dict to retrieve the name from the number at the end
label_dict = dict(zip(persons, labels))


#helper function to retrieve a name from a given label
def get_person_name_from_label(dict, search_label):
    for name, label in dict.items():
        if label == search_label:
            return name
    return None


#helper function to find most frequent element in a list
#returns the whole dict as well as the most frequent element
def most_frequent(array):
    occurence_count = Counter(array)
    return occurence_count, occurence_count.most_common(1)[0][0] 


root = Tk()                
root.title("Face recognition")
root.geometry('400x200')

was_repeated = False

recognized_list = []

def recognize(image_size = 96, counter = 15):
    result_label.config(text='')
    camera = cv2.VideoCapture(0)
    
    while counter > 0:
        ret, frame = camera.read()
        if ret:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #change a color photo to gray
            faces = face_cascade.detectMultiScale(image, 1.3, 5)
            for (x, y, width, height) in faces:
                roi_color = image[y:y + height, x:x + width]
                face = cv2.resize(roi_color, (image_size, image_size))
                #CNN used a conv2d that required 4-dimensions: (images x height x width x channel)
                #changing dimension to 1 x image_size x image_size x 1        
                img_array = np.asarray(face) / 255.0
                img_array = img_array.reshape(1, img_array.shape[0], img_array.shape[1], 1)

                #prediction
                params = model.predict(img_array)
                label = np.argmax(params[0], -1) #get predicted label
                value = params[0][label] #get probability for the predicted label

                if value > 0.7: #if probability of accurance is > 70%
                    name = get_person_name_from_label(label_dict, label)
                    recognized_list.append(name)
                else: #not recognized
                    recognized_list.append("")
                    name = "Access denied"
                cv2.putText(frame, name, (x, y - 20), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                cv2.rectangle(frame, (x, y), (x + width, y + height), (255,0,0), 2)
                counter -= 1


            cv2.imshow('frame', frame)
            if cv2.waitKey(1) == ord('q'):
                break

    
    camera.release()
    cv2.destroyAllWindows()
    
    #evalute the person's name
    occurence_dir, most_common = most_frequent(recognized_list)
    
    if most_common == "" or occurence_dir[most_common] / 15 < 0.9: #not recognized at all or different results
        result_label.config(text='Unrecognized - no access'.upper(), 
                            fg='red', 
                            font=("Arial", 12)
                           )
        
        button.pack_forget() #delete the button
        button.pack() #show the button again (to place it below the label)
        button.config(text="RETRY", fg='black') #change the button's text
        
        if was_repeated: #only one chance to repeat recognition possible
            button.pack_forget()

    else:
        button.pack_forget()
        result_label.config(text='Recognized: {} - access granted'.format(most_common).upper(), 
                            fg='green', 
                            font=("Arial", 12)
                           )
    

window_frame = Frame(root)

button = Button(window_frame, text="Click to start recognizing".upper(), command=recognize, fg='green')
result_label = Label(window_frame, text="")

window_frame.pack(expand=True) #expand assigns additional space to the frame if the parent is expanded
button.pack()
result_label.pack()
    

root.mainloop()